In [2]:
import io
import os
import re
import keras
import tensorflow as tf
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#kw=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/temp_frequent.txt',header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
input_dim=5000
count_vect = CountVectorizer(max_features = input_dim)
X_train_counts = count_vect.fit_transform(docs)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
X_train_tfidf = X_train_tfidf.toarray()
X_train_tfidf = np.reshape(X_train_tfidf, (X_train_tfidf.shape[0], 1, -1))
print(X_train_tfidf.shape)

(91795, 1, 5000)


In [0]:

kw.columns=['words']
dfreq={}
for k in kw['words']:
  dfreq[k]=0
print(len(dfreq))

In [0]:
for d in docs:
  words=re.split('[\s]',d)
  words=np.unique(words)
  for w in words:
    if w in dfreq:
      dfreq[w]=dfreq[w]+1

tf_idf = []
for d in docs:
  words=re.split('[\s]',d)
  N=len(words)
  tfreq={}
  for w in words:
    if w in tfreq:
      tfreq[w]=tfreq[w]+1
    else:
      tfreq[w]=1
  temp=[]
  for w in dfreq:
    if w in tfreq:
      tf=tfreq[w]*1.0/N
      idf=np.log(len(docs)*1.0/dfreq[w])
      temp.append(tf*idf)
    else:
      temp.append(0.0)
  tf_idf.append(temp)

tf_idf = np.array(tf_idf)
input_dim=len(tf_idf[0])
print(input_dim)
tf_idf = np.reshape(tf_idf, (tf_idf.shape[0], 1, tf_idf.shape[1]))

19999


In [6]:

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
print(labels)
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y)


['science_tech' 'state' 'arts_literature' ... 'international' 'sports'
 'health_lifestyle']
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, dummy_y, test_size=0.18)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 5000)
(75271, 8)
(16524, 1, 5000)
(16524, 8)


In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(2048, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
model.add(LSTM(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3))
model.add(LSTM(512,return_sequences=True,dropout=0.4,recurrent_dropout=0.3))
model.add(LSTM(128,return_sequences=True,dropout=0.4,recurrent_dropout=0.3))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(8, activation='softmax'))

In [0]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

In [10]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256,shuffle=False, verbose=1,validation_split=0.1)
#model.save_weights('./checkpoints/text_classifier_rnn_tfidf')
!mkdir -p saved_model
model.save('saved_model/text_classifier_rnn_tfidf')

Train on 67743 samples, validate on 7528 samples
Epoch 1/25
67743/67743 [==============================] - 20s 300us/step - loss: 1.5376 - acc: 0.3713 - val_loss: 0.8629 - val_acc: 0.7448
Epoch 2/25
67743/67743 [==============================] - 16s 241us/step - loss: 0.5304 - acc: 0.8409 - val_loss: 0.3819 - val_acc: 0.8925
Epoch 3/25
67743/67743 [==============================] - 16s 242us/step - loss: 0.2991 - acc: 0.9174 - val_loss: 0.3178 - val_acc: 0.9067
Epoch 4/25
67743/67743 [==============================] - 16s 240us/step - loss: 0.2313 - acc: 0.9365 - val_loss: 0.3224 - val_acc: 0.9099
Epoch 5/25
67743/67743 [==============================] - 16s 242us/step - loss: 0.1921 - acc: 0.9475 - val_loss: 0.3306 - val_acc: 0.9123
Epoch 6/25
67743/67743 [==============================] - 16s 240us/step - loss: 0.1695 - acc: 0.9532 - val_loss: 0.3462 - val_acc: 0.9094
Epoch 7/25
67743/67743 [==============================] - 16s 239us/step - loss: 0.1526 - acc: 0.9586 - val_loss: 0.3

In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_tfidf')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_tfidf')

In [11]:
results = model.evaluate(X_test, y_test)
print(results)

16524/16524 [==============================] - 2s 140us/step
[0.535043280980435, 0.9050471782684326]


In [0]:
tf_idf.shape

(17687, 1, 19999)

In [0]:
y_pred=model.predict(X_test)

In [15]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [17]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(prec)
print(rec)
print(f_score)

[90.05681818 88.68544601 91.10132159 91.81875393 88.00925926 91.11483655
 96.05100782 87.68472906]
[95.62594268 89.18791313 87.47884941 92.16677195 88.37749884 88.37398374
 95.38398693 87.61832639]
[92.75786394 88.93596987 89.25334484 91.9924338  88.19299466 89.72348329
 95.71633531 87.65151515]
